In [1]:
import os
import numpy as np

In [2]:
basedir = "/home/timeTravelling/yuanke/nerf-pytorch/reconstruct_output"

In [3]:
poses_arr = np.load(os.path.join(basedir, 'poses_bounds.npy'))

In [4]:
poses = poses_arr[:, :-2].reshape([-1, 3, 5]).transpose([1,2,0])
poses.shape

(3, 5, 100)

In [5]:
poses = np.moveaxis(poses, -1, 0).astype(np.float32)

In [6]:
poses.shape

(100, 3, 5)

In [9]:
H, W, K, c2w =800,800,0,poses[0,:3,:4]

In [10]:
i, j = np.meshgrid(np.arange(W, dtype=np.float32), np.arange(H, dtype=np.float32), indexing='xy')

In [12]:
k = np.meshgrid(np.arange(W, dtype=np.float32), np.arange(H, dtype=np.float32), indexing='xy')

In [14]:
hwf = poses[0,:3,-1]
H, W, focal = hwf
H, W = int(H), int(W)
hwf = [H, W, focal]
K = np.array([
    [focal, 0, 0.5*W],
    [0, focal, 0.5*H],
    [0, 0, 1]
])

In [16]:
dirs = np.stack([(i-K[0][2])/K[0][0], -(j-K[1][2])/K[1][1], -np.ones_like(i)], -1)

In [25]:
rays_d = np.sum(dirs[..., np.newaxis, :] * c2w[:3,:3], -1)

In [29]:
poses.shape

(100, 3, 5)

In [30]:
def get_rays_np(H, W, K, c2w):
    i, j = np.meshgrid(np.arange(W, dtype=np.float32), np.arange(H, dtype=np.float32), indexing='xy')
    dirs = np.stack([(i-K[0][2])/K[0][0], -(j-K[1][2])/K[1][1], -np.ones_like(i)], -1)
    rays_d = np.sum(dirs[..., np.newaxis, :] * c2w[:3,:3], -1)  # dot product, equals to: [c2w.dot(dir) for dir in dirs]
    rays_o = np.broadcast_to(c2w[:3,-1], np.shape(rays_d))
    return rays_o, rays_d
rays = np.stack([get_rays_np(H, W, K, p) for p in poses[:,:3,:4]], 0)

In [31]:
rays.shape

(100, 2, 800, 800, 3)

In [2]:
import torch

In [3]:
t_vals = torch.linspace(0., 1., steps=64)

In [4]:
t_vals.shape

torch.Size([64])

In [7]:
near, far = 0 * torch.ones_like(torch.Tensor(1024)), 1 * torch.ones_like(torch.Tensor(1024))

In [12]:
near, far = near.reshape(1024,1), far.reshape(1024,1)

In [13]:
z_vals = near * (1.-t_vals) + far * (t_vals)

In [16]:
z_vals = z_vals.expand([1024, 64])

In [18]:
z_vals

tensor([[0.0000, 0.0159, 0.0317,  ..., 0.9683, 0.9841, 1.0000],
        [0.0000, 0.0159, 0.0317,  ..., 0.9683, 0.9841, 1.0000],
        [0.0000, 0.0159, 0.0317,  ..., 0.9683, 0.9841, 1.0000],
        ...,
        [0.0000, 0.0159, 0.0317,  ..., 0.9683, 0.9841, 1.0000],
        [0.0000, 0.0159, 0.0317,  ..., 0.9683, 0.9841, 1.0000],
        [0.0000, 0.0159, 0.0317,  ..., 0.9683, 0.9841, 1.0000]])

In [20]:
mids = .5 * (z_vals[...,1:] + z_vals[...,:-1])
mids.shape

torch.Size([1024, 63])

In [21]:
upper = torch.cat([mids, z_vals[...,-1:]], -1)
upper.shape

torch.Size([1024, 64])

In [22]:
lower = torch.cat([z_vals[...,:1], mids], -1)

In [23]:
t_rand = torch.rand(z_vals.shape)

In [24]:
t_rand

tensor([[0.6841, 0.5793, 0.1013,  ..., 0.7408, 0.6742, 0.6534],
        [0.6035, 0.6115, 0.7997,  ..., 0.4922, 0.0203, 0.9313],
        [0.5335, 0.8111, 0.9613,  ..., 0.2066, 0.5531, 0.9985],
        ...,
        [0.5172, 0.1228, 0.4495,  ..., 0.7886, 0.3606, 0.9098],
        [0.6594, 0.2136, 0.9326,  ..., 0.9645, 0.4376, 0.4564],
        [0.4754, 0.6192, 0.4316,  ..., 0.7974, 0.9276, 0.3128]])

In [28]:
(upper[...,None,:] + lower[...,None,:] + t_rand[...,:,None]).shape

torch.Size([1024, 64, 64])

In [33]:
(lower[...,None,:3] * t_rand[...,:,None]).shape

torch.Size([1024, 64, 3])

In [30]:
lower

tensor([[0.0000, 0.0079, 0.0238,  ..., 0.9603, 0.9762, 0.9921],
        [0.0000, 0.0079, 0.0238,  ..., 0.9603, 0.9762, 0.9921],
        [0.0000, 0.0079, 0.0238,  ..., 0.9603, 0.9762, 0.9921],
        ...,
        [0.0000, 0.0079, 0.0238,  ..., 0.9603, 0.9762, 0.9921],
        [0.0000, 0.0079, 0.0238,  ..., 0.9603, 0.9762, 0.9921],
        [0.0000, 0.0079, 0.0238,  ..., 0.9603, 0.9762, 0.9921]])

In [31]:
upper[...,None,:] + lower[...,None,:]

tensor([[[0.0079, 0.0317, 0.0635,  ..., 1.9365, 1.9683, 1.9921]],

        [[0.0079, 0.0317, 0.0635,  ..., 1.9365, 1.9683, 1.9921]],

        [[0.0079, 0.0317, 0.0635,  ..., 1.9365, 1.9683, 1.9921]],

        ...,

        [[0.0079, 0.0317, 0.0635,  ..., 1.9365, 1.9683, 1.9921]],

        [[0.0079, 0.0317, 0.0635,  ..., 1.9365, 1.9683, 1.9921]],

        [[0.0079, 0.0317, 0.0635,  ..., 1.9365, 1.9683, 1.9921]]])

In [34]:
freq_bands = 2.**torch.linspace(0., 9, steps=10)

In [36]:
x = torch.Tensor([[1,2,3],[4,5,6]])

In [38]:
embed_fns = []

In [39]:
for freq in freq_bands:
    for p_fn in [torch.sin, torch.cos]:
        embed_fns.append(p_fn(x * freq))

In [48]:
embed_fns

[tensor([[ 0.8415,  0.9093,  0.1411],
         [-0.7568, -0.9589, -0.2794]]),
 tensor([[ 0.5403, -0.4161, -0.9900],
         [-0.6536,  0.2837,  0.9602]]),
 tensor([[ 0.9093, -0.7568, -0.2794],
         [ 0.9894, -0.5440, -0.5366]]),
 tensor([[-0.4161, -0.6536,  0.9602],
         [-0.1455, -0.8391,  0.8439]]),
 tensor([[-0.7568,  0.9894, -0.5366],
         [-0.2879,  0.9129, -0.9056]]),
 tensor([[-0.6536, -0.1455,  0.8439],
         [-0.9577,  0.4081,  0.4242]]),
 tensor([[ 0.9894, -0.2879, -0.9056],
         [ 0.5514,  0.7451, -0.7683]]),
 tensor([[-0.1455, -0.9577,  0.4242],
         [ 0.8342, -0.6669, -0.6401]]),
 tensor([[-0.2879,  0.5514, -0.7683],
         [ 0.9200, -0.9939,  0.9836]]),
 tensor([[-0.9577,  0.8342, -0.6401],
         [ 0.3919, -0.1104, -0.1804]]),
 tensor([[ 0.5514,  0.9200,  0.9836],
         [ 0.7210,  0.2194, -0.3549]]),
 tensor([[ 0.8342,  0.3919, -0.1804],
         [-0.6929, -0.9756, -0.9349]]),
 tensor([[ 0.9200,  0.7210, -0.3549],
         [-0.9992, -0.4282

In [47]:
freq_bands.shape

torch.Size([10])

In [49]:
(1024,64,3)[:-1]

(1024, 64)

In [2]:
import torch
x = torch.zeros(1024, 2, 3)

In [5]:
import numpy as np

In [7]:
rays_rgb = np.zeros((1024, 3, 3))

In [9]:
rays_rgb = torch.Tensor(rays_rgb)
batch = torch.transpose(rays_rgb, 0, 1)
batch.shape

torch.Size([3, 1024, 3])

In [10]:
batch_rays = batch[:2]

In [11]:
rays_o, rays_d = batch_rays

In [12]:
rays_d.shape

torch.Size([1024, 3])